# Задание 6

### Пример входных данных

In [25]:
import pandas as pd
clinton = pd.read_csv("clinton.csv", encoding="utf-8")
trump = pd.read_csv("trump.csv", encoding="utf-8")

In [2]:
trump['text'][0]

'Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc'

In [3]:
str(trump.loc[0][0])

'Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc'

In [4]:
clinton.head()

,text
0,The question in this election: Who can put the...
1,"Last night, Donald Trump said not paying taxes..."
2,Couldn't be more proud of @HillaryClinton. Her...
3,"If we stand together, there's nothing we can't..."
4,Both candidates were asked about how they'd co...


## Описание задания

Предлагается написать генератор случайных диалогов на основе твитов Трампа (файл <i>"trump.csv"</i>) и Хеллори Клинтон (файл <i>"clinton.csv"</i>).

* Каждый диалог состоит из нескольких "ходов" (<i>turn</i>).
* Каждый такой <i>turn</i> представляет собой цепочку нескольких реплик разных агентов (участников диалога, их может быть больше двух).
* Цепочка реплик представляет собой набор реплик агентов. Первое сообщение в цепочке - <i>"исходное сообщение"</i> на которое отвечают оставшиеся агенты (по одному сообщению за turn).

Программа должна представлять отдельный пакет следующей структуры:
<ul>
<li>run\_generator.py</li>
<li>dialogsgenerator:
<ul>
<li>agent.py</li>
<li>randomdialog.py</li>
<li>\_\_init__.py</li>
</ul></li>
</ul>

<b>URGENT:</b> в работе ЗАПРЕЩАЕТСЯ использовать циклы:
<ul>
<li>Использование 1 for (или while): max 1 балл</li>
<li>Использование до 4 for (или while): max 0.8 баллов</li>
<li>Использование 4+ for (или while): max 0.5 баллов</li>
</ul>

### Файл agent.py

Описание класса агента (один класс для всех агентов, в нашем случае для Трампа и Клинтон).

In [22]:
from random import randint

In [63]:
from collections import Generator


class Agent(Generator):
    
    def __init__(self, kb, name):
        # Инициализация
        self.tweets_pool = kb['text']
        self.num_tweets = len(kb)
        print("i'm in init")
        self.name = name

    def send(self, msg):
        # Необходимо написать свой собственный метод send для генератора
#         msg = yield
        random_idx = randint(0, self.num_tweets - 1)
        print("agent name is ", self.name)
        return self.tweets_pool[random_idx]

    def throw(self, typ=None, val=None, tb=None):
        # Оставляем как есть
        super().throw(typ, val, tb)

    def __str__(self):
        # Опишем строковое представление класса
        return self.name

    def __repr__(self):
        return str(self)

#### Пример использования:

In [64]:
# from dialogsgenerator import Agent
clint, trum = Agent(clinton, "clinton"), Agent(trump, "trump")
clint.send("hello")
# type(clint)

i'm in init
i'm in init
agent name is  clinton


'Donald Trump called the Trump Plaza Casino and Hotel "the biggest hit yet."\n\nNow, it\'s abandoned. https://t.co/qjhy8Xj2U8'

In [65]:
# print(next(clint))
# print(next(clint))
# print(next(clint))
# print(next(clint))

### Файл randomdialog.py

Описание класса, генерирующего случайный диалог.

In [66]:
import sys

In [96]:
class RandomDialog(object):
    
    def __init__(self, agents, max_len=5):
        # Инициализация
        self.agents = agents
        self.agents_num = len(agents)
        self.max_len = max_len

    def generate(self):
        """
        Генерирует случайный диалог состоящий из 1-max_len цепочек.
        При генерации вызывает функцию turn.
        Возвращаемый объект является генератором.
        """
#         rand_agent_ind = randint(0, )
        pass


    def turn(self):
        """
        Генерирует одну случайную цепочку следующим образом: выбирается случайный агент.
        Он "говорит" случайное сообщение (msg) из своей Agent.kb (используйте next или send(None)).
        (правила того, как выбирать никак не регулируются, вы можете выбирать случайный твит из Agent.kb никак не учитывая
        переданное msg).
        Это сообщение передается с помощью send (помним, что агент - это объект-генератор).
        Далее получаем ответ от всех агентов и возвращаем (генерируем) их (включая исходное сообщение).
        Возвращаемый объект является генератором.
        """
        #<your code here>
        rand_agent_ind = randint(0, self.agents_num - 1)
        msg = next(self.agents[rand_agent_ind])
        it = iter(self.agents)
        print(list(map(print, [it]*self.agents_num)))

    def eval(self, dialog=None):
        """
        Превращает генератор случайного далога (который возвращается в self.generate())
        в список списков (пример использования далее).
        Возвращаемый объект является списком.
        """
        if dialog is None:
            dialog = self.generate()
#         <your code here>

    def write(self, dialog=None, target=sys.stdout):
        """
        Записывает результат self.eval() в соответствующий поток.
        """
        if dialog is None:
            dialog = self.eval()
#         <your code here>

    pass


In [97]:
test_random_diaog = RandomDialog([Agent(clinton, "clinton"), Agent(trump, "trump")], max_len=2)

i'm in init
i'm in init


In [98]:
test_random_diaog.turn()

agent name is  trump
[None, None]


#### Пример использования:

In [854]:
# from dialogsgenerator import RandomDialog
rd = RandomDialog([Agent(clinton, "clinton"), Agent(trump, "trump")], max_len=2)

In [855]:
generated = rd.generate()
generated

<generator object RandomDialog.generate at 0x0000007DC85901A8>

In [856]:
evaled = rd.eval(generated)
evaled

[["clinton: Let's make it easier for young people to start businesses and fulfill their dreams. https://t.co/cJzNCnaahh https://t.co/vZHW9VZKDT",
  'trump: My list of potential U.S. Supreme Court Justices was very well recieved. During the next number of weeks I may be adding to the list!'],
 ["trump: DON'T LET HILLARY CLINTON DO IT AGAIN!\r #TrumpPence16\r https://t.co/1mGkPNZPKF",
  'clinton: With the first local transmission of Zika in FL, it’s even more critical for Republicans to stop blocking action: https://t.co/KjmWMDOgDl']]

In [857]:
rd.write(evaled)


turn 0:
	clinton: Let's make it easier for young people to start businesses and fulfill their dreams. https://t.co/cJzNCnaahh https://t.co/vZHW9VZKDT
	trump: My list of potential U.S. Supreme Court Justices was very well recieved. During the next number of weeks I may be adding to the list!
turn 1:
 https://t.co/1mGkPNZPKF
	clinton: With the first local transmission of Zika in FL, it’s even more critical for Republicans to stop blocking action: https://t.co/KjmWMDOgDl

### Файл run_generator.py

Содержит функции для генерации и записи нескольких диалогов. Файл должен быть написан так, чтобы его можно было запускать через командную строку:
<img src="cmd.png">
Описание аргументов представлено на рисунке:
<img src="cmd_help.png">
Для разбора аргументов ипользуйте <a href="https://docs.python.org/3/howto/argparse.html">модуль argparse</a>. Все представленные на рисунке аргументы должны быть обработаны (кроме help, он обрабатывается автоматически модулем argparse).

In [846]:
def generate(rd, count_dialogs=5):
    """
    Генерирует count_dialogs диалогов с помощью rd.generate().
    Возвращаемый объект является генератором.
    """
    <your code here>

def write(dialogs, target):
    """
    Записывает сгенерированные диалоги dialogs (это объект-генератор) в поток target.
    """
    <your code here>


if __name__ == "__main__":
    <your code here>


#### Пример использования функций из файла:

In [869]:
import sys
from run_generator import generate, write

In [870]:
dialogs = generate(rd, 2)
dialogs

In [871]:
write(dialogs, sys.stdout)

________________________________________Dialog 0________________________________________
turn 0:
	trump: Crooked Hillary wants to take your 2nd Amendment rights away. Will guns be taken from her heavily armed Secret Service detail? Maybe not!
	clinton: "Bill, that conversation we started in the law library 45 years ago, it is still going strong.” —Hillary
turn 1:
	clinton: Our teachers deserve more than just a pat on the back. They deserve a raise. #TeacherAppreciationDay https://t.co/0sgmUFg6mX
	trump: I visited our Trump Tower campaign headquarters last night, after returning from Ohio and Arizona, and it was packed with great pros - WIN!
________________________________________Dialog 1________________________________________
turn 0:
	clinton: "He won me over with that Mexican rapist speech." —Ann Coulter on Donald Trump https://t.co/GTNOhKYMA2
 #MakeAmericaGreatAgain #Trump2016 https://t.co/7WHS3vbXSw
turn 1:
	trump: .@FreeJesseJames- Just read your complete statement. You are an am